In [ ]:
import numpy as np # linear algebra
import pandas as pd
import os
from tqdm import tqdm
import shutil

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')
test = pd.read_csv('../input/shopee-product-matching/test.csv')
sample = pd.read_csv('../input/shopee-product-matching/sample_submission.csv')

In [ ]:
import os
import json
import math
from tqdm import tqdm 

import cv2
import numpy as np
import pandas as pd
from keras import layers, optimizers
from keras.applications import MobileNetV2
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential
import cv2
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

**CNN***

In [ ]:
generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2,
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True,  # randomly flip images
    preprocessing_function=tf.keras.applications.resnet.preprocess_input
    )
testgenerator = ImageDataGenerator(rescale = 1./255)
target_size = [224, 224]
batch_size = 32
train_dir='../input/test1/kaggle/working/class/'
train_gen = generator.flow_from_directory(train_dir,
 target_size = target_size,
 batch_size = batch_size,
 shuffle=True,
 class_mode = 'categorical',                                        
 subset='training')

val_gen = generator.flow_from_directory(train_dir,
 target_size = target_size,
 batch_size = batch_size,
 shuffle=True,
 class_mode = 'categorical',                                        
 subset='validation')

In [ ]:
# base_model=MobileNet(weights='imagenet',include_top=False,input_shape=(224,224,3)) #imports the mobilenet model and discards the last 1000 neuron layer.

# x=base_model.output
# x=GlobalAveragePooling2D()(x)
# x=Dense(224,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
# x=Dense(224,activation='relu')(x) #dense layer 2
# x=Dense(128,activation='relu')(x) #dense layer 3
# preds=Dense(11014,activation='softmax')(x) #final layer with softmax activation

# model=Model(inputs=base_model.input,outputs=preds)

In [ ]:
# model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['AUC'])

In [ ]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]


    return img_tensor

    
img_path = '../input/shopee-product-matching/test_images'

#tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
posting_id=test['posting_id'].tolist()
# print(sample_sub)

labels = (train_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
preds=[]

model=tf.keras.models.load_model('../input/model-10e/my_test1_model.h5')
for test_image in test['image']:
        #predicted_class_indices=[]
        testimg=load_image(os.path.join(img_path,test_image))
        preds.append(model.predict(testimg))
label_group=[]        
for pred in preds:
    predicted_class_indices=np.argmax(pred,axis=1)
    predictions = [labels[k] for k in predicted_class_indices]    
    label_group.append(predictions)    
print(label_group)

sample_sub = pd.DataFrame(posting_id, columns=['posting_id'])
sample_sub = pd.concat([sample_sub, pd.DataFrame(label_group,columns=['label_group'])],axis=1)

 

        
# sample_sub['matches']=sample_sub.label_group.map(tmp)
sample_sub.head()

In [ ]:
train.head()

In [ ]:
t=load_image('../input/shopee-product-matching/train_images/0000a68812bc7e98c42888dfb1c07da0.jpg')
x=model.predict(t)

In [ ]:
tp=np.argmax(x,axis=1)
predict = [labels[k] for k in tp]  

In [ ]:
predict

In [ ]:
sample_sub['label_group']=pd.DataFrame(sample_sub['label_group'],dtype=np.int)

In [ ]:
tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
sample_sub['matches']=sample_sub.label_group.map(tmp)

sample_sub.head()

In [ ]:
tmp2 = test.groupby('image_phash').posting_id.agg('unique').to_dict()
test['pred_phash']=test.image_phash.map(tmp2)
test

In [ ]:
submission=sample_sub.drop(['label_group'], axis=1)
submission

In [ ]:
for i in range(len(submission['matches'])):
    submission['matches'][i]=submission['matches'][i].tolist()

In [ ]:
submission['matches']

In [ ]:
for i in range(len(submission['posting_id'])):
    submission['matches'][i].append(test['posting_id'][i])

In [ ]:
def clean(x):
    return " ".join(x)
submission['matches']=submission.matches.map(clean)
submission

In [ ]:
type(sample['matches'])

In [ ]:
sample.info()

In [ ]:
submission.info()

In [ ]:
type(submission['matches'])

**imagephash**

**TFIDF***

In [ ]:
submission.to_csv('submission.csv', index=False)